In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = './data/'

In [3]:
currency_pair = 'Nzd_Jpy'

In [4]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_2021-2022.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [5]:
df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume
0,2021-01-03 22:00:00,74.110,74.188,74.096,74.177,74.233,74.355,74.210,74.276,74.172,74.260,74.156,74.226,58
1,2021-01-03 22:05:00,74.179,74.209,74.178,74.192,74.276,74.305,74.275,74.277,74.228,74.250,74.227,74.234,47
2,2021-01-03 22:10:00,74.191,74.209,74.189,74.190,74.278,74.282,74.276,74.276,74.234,74.244,74.232,74.233,20
3,2021-01-03 22:15:00,74.189,74.190,74.187,74.187,74.276,74.277,74.276,74.277,74.232,74.233,74.232,74.232,7
4,2021-01-03 22:20:00,74.190,74.202,74.172,74.190,74.274,74.304,74.273,74.273,74.232,74.244,74.231,74.232,28


In [6]:
df.tail()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume
74788,2021-12-31 21:35:00,78.697,78.725,78.684,78.725,78.742,78.768,78.726,78.768,78.720,78.746,78.706,78.746,24
74789,2021-12-31 21:40:00,78.723,78.730,78.713,78.716,78.765,78.775,78.755,78.767,78.744,78.752,78.736,78.742,39
74790,2021-12-31 21:45:00,78.720,78.750,78.718,78.750,78.765,78.799,78.765,78.799,78.742,78.774,78.742,78.774,50
74791,2021-12-31 21:50:00,78.751,78.793,78.750,78.780,78.799,78.840,78.799,78.821,78.775,78.816,78.775,78.800,60
74792,2021-12-31 21:55:00,78.780,78.787,78.590,78.661,78.825,78.930,78.815,78.902,78.802,78.830,78.716,78.782,273


In [7]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [8]:
rounding = 5 if 'Jpy' not in currency_pair else 3
divider = 10000 if 'Jpy' not in currency_pair else 100

In [9]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, each_bar, n_bars, pip_movement, use_pullback):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    trade = None
    lookback = n_bars + 1 if use_pullback else n_bars
    lookforward = -1 if use_pullback else 0
    pip_movement /= divider

    for i in range(lookback, len(df)):
        curr_bid_open, curr_bid_high, curr_bid_low, curr_ask_open, curr_ask_high, curr_ask_low, curr_mid_open, curr_date = df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Mid_Open', 'Date']]
        spread = abs(curr_ask_open - curr_bid_open)

        mid_opens = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Open'])
        mid_highs = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_High'])
        mid_lows = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Low'])
        mid_closes = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Close'])

        if each_bar:
            buy_signal = all([mid_opens[j] < mid_closes[j] and abs(mid_opens[j] - mid_closes[j]) >= pip_movement for j in range(len(mid_opens))])
            sell_signal = all([mid_opens[j] > mid_closes[j] and abs(mid_opens[j] - mid_closes[j]) >= pip_movement for j in range(len(mid_opens))])

        else:
            buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement
            sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement

        if use_pullback and buy_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            buy_signal = mid_open1 > mid_close1 and abs(mid_close1 - mid_open1) <= 0.25 * abs(mid_high1 - mid_low1)

        if use_pullback and sell_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            sell_signal = mid_open1 < mid_close1 and abs(mid_close1 - mid_open1) <= 0.25 * abs(mid_high1 - mid_low1)

        highest_high, lowest_low = max(mid_highs), min(mid_lows)

        if trade is None:
            if buy_signal:
                open_price = float(curr_ask_open)
                stop_loss = round(lowest_low, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

            elif sell_signal:
                open_price = float(curr_bid_open)
                stop_loss = round(highest_high, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)


        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None    

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            # trade_amount = 50.0 * risk_reward_ratio
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [10]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.0, 1.5]
spread_cutoffs = [0.10]
each_bar_vals = [False]
n_bars_vals = [3, 4, 5]
pip_movement_vals = [20, 30]
use_pullback_vals = [True]

n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(each_bar_vals) * len(n_bars_vals) * len(pip_movement_vals) * len(use_pullback_vals)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for each_bar in each_bar_vals:
            for n_bars in n_bars_vals:
                for pip_movement in pip_movement_vals:
                    for use_pullback in use_pullback_vals:
                        all_combos.append((risk_reward_ratio, spread_val, each_bar, n_bars, pip_movement, use_pullback))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_each_bar_val = None
best_n_bars_val = None
best_pip_movement_val = None
best_use_pullback_val = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, each_bar, n_bars, pip_movement, use_pullback in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, each_bar, n_bars, pip_movement, use_pullback)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_sells))
    print('Num sells: ' + str(n_buys))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'each_bar': each_bar, 'n_bars': n_bars, 'pip_movement': pip_movement, 'use_pullback': use_pullback})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_each_bar_val = each_bar
        best_n_bars_val = n_bars
        best_pip_movement_val = pip_movement
        best_use_pullback_val = use_pullback

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 12

-1.8192
Num buys: 1
Num sells: 1
Num trades: 2
Num wins: 1
Num losses: 1
Win streak: 1
Loss streak: 1
Avg pips risked: 0.4399999999999977
Remaining runs: 11
Best reward so far: -1.8192

98.48544
Num buys: 2
Num sells: 0
Num trades: 2
Num wins: 2
Num losses: 0
Win streak: 2
Loss streak: 0
Avg pips risked: 0.41499999999999915
Remaining runs: 10
Best reward so far: 98.48544

119.2754
Num buys: 6
Num sells: 4
Num trades: 10
Num wins: 5
Num losses: 5
Win streak: 3
Loss streak: 3
Avg pips risked: 0.29269999999999785
Remaining runs: 9
Best reward so far: 119.2754

195.71936
Num buys: 6
Num sells: 4
Num trades: 10
Num wins: 7
Num losses: 3
Win streak: 3
Loss streak: 2
Avg pips risked: 0.29269999999999785
Remaining runs: 8
Best reward so far: 195.71936

-101.6561
Num buys: 1
Num sells: 1
Num trades: 2
Num wins: 0
Num losses: 2
Win streak: 0
Loss streak: 2
Avg pips risked: 0.4399999999999977
Remaining runs: 7
Best reward so far: 195.71936

-3.0469600000000003
Num buys: 7
Num sells:

In [11]:
# GBP_JPY
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best each bar val: ' + str(best_each_bar_val))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 22206.268699998527
Best risk/reward ratio: 1.0
Best spread: 0.1
Best each bar val: False
Best n bars val: 3
Best pip movement val: 20
Best use pullback val: True
-----------------------
Top results:
{'reward': 22206, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True}
{'reward': -28071, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 30, 'use_pullback': True}
{'reward': -7815, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 30, 'use_pullback': True}
{'reward': -39013, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 30, 'use_pullback': True}
{'reward': -61281, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 4, 'pip_movement': 20, 'use_pullback': True}
{'reward': -14876, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True}
{'reward': -30825,

In [11]:
# EUR_JPY
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best each bar val: ' + str(best_each_bar_val))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 12737.032280000207
Best risk/reward ratio: 1.0
Best spread: 0.1
Best each bar val: False
Best n bars val: 4
Best pip movement val: 20
Best use pullback val: True
-----------------------
Top results:
{'reward': -6581, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 30, 'use_pullback': True}
{'reward': -6395, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 4, 'pip_movement': 30, 'use_pullback': True}
{'reward': -13288, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True}
{'reward': -19344, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 4, 'pip_movement': 20, 'use_pullback': True}
{'reward': -26144, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 30, 'use_pullback': True}
{'reward': 3465, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 4, 'pip_movement': 30, 'use_pullback': True}
{'reward': -3351, 'r

In [11]:
# NZD_JPY
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best each bar val: ' + str(best_each_bar_val))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 296.43516
Best risk/reward ratio: 1.0
Best spread: 0.1
Best each bar val: False
Best n bars val: 4
Best pip movement val: 20
Best use pullback val: True
-----------------------
Top results:
{'reward': -1, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 30, 'use_pullback': True}
{'reward': 98, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 4, 'pip_movement': 30, 'use_pullback': True}
{'reward': 119, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 20, 'use_pullback': True}
{'reward': 195, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 20, 'use_pullback': True}
{'reward': -3, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True}
{'reward': 296, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 4, 'pip_movement': 20, 'use_pullback': True}
{'reward': -78, 'ratio': 1.5, 'spread': 0.1, '

In [13]:
# AUD_JPY
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best each bar val: ' + str(best_each_bar_val))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 10421.13232000061
Best risk/reward ratio: 1.5
Best spread: 0.1
Best each bar val: False
Best n bars val: 3
Best pip movement val: 20
Best use pullback val: True
-----------------------
Top results:
{'reward': 0, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 4, 'pip_movement': 50, 'use_pullback': True}
{'reward': 4231, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 50, 'use_pullback': True}
{'reward': 10421, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True}
{'reward': 2211, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 40, 'use_pullback': True}
{'reward': 6344, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 50, 'use_pullback': True}
{'reward': 6363, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 30, 'use_pullback': True}
{'reward': 176, 'ratio': 1.0, '

In [11]:
# USD_JPY
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best each bar val: ' + str(best_each_bar_val))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 73419.96127999948
Best risk/reward ratio: 1.5
Best spread: 0.1
Best each bar val: False
Best n bars val: 5
Best pip movement val: 20
Best use pullback val: True
-----------------------
Top results:
{'reward': 11363, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 40, 'use_pullback': True}
{'reward': 11063, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 30, 'use_pullback': True}
{'reward': 5605, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 4, 'pip_movement': 20, 'use_pullback': True}
{'reward': 24886, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True}
{'reward': 5777, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 40, 'use_pullback': True}
{'reward': 73419, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 20, 'use_pullback': True}
{'reward': 4964, 'ratio':

In [11]:
# USD_CAD
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best each bar val: ' + str(best_each_bar_val))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 846.4951200000075
Best risk/reward ratio: 1.5
Best spread: 0.1
Best each bar val: False
Best n bars val: 3
Best pip movement val: 30
Best use pullback val: True
-----------------------
Top results:
{'reward': 384, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True}
{'reward': 357, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 30, 'use_pullback': True}
{'reward': 300, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 40, 'use_pullback': True}
{'reward': 810, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True}
{'reward': 253, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 20, 'use_pullback': True}
{'reward': 837, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 20, 'use_pullback': True}
{'reward': 210, 'ratio': 1.5, 'spre

In [11]:
# AUD_USD
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best each bar val: ' + str(best_each_bar_val))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 358.2074999999962
Best risk/reward ratio: 1.5
Best spread: 0.1
Best each bar val: False
Best n bars val: 4
Best pip movement val: 20
Best use pullback val: True
-----------------------
Top results:
{'reward': 72, 'ratio': 0.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True}
{'reward': 46, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 25, 'use_pullback': True}
{'reward': 44, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True}
{'reward': 117, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True}
{'reward': 73, 'ratio': 0.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 4, 'pip_movement': 25, 'use_pullback': True}
{'reward': 49, 'ratio': 0.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 4, 'pip_movement': 20, 'use_pullback': True}
{'reward': 191, 'ratio': 1.0, 'spread': 

In [11]:
# EUR_USD
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best each bar val: ' + str(best_each_bar_val))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 672.7452700000051
Best risk/reward ratio: 1.5
Best spread: 0.1
Best each bar val: False
Best n bars val: 3
Best pip movement val: 20
Best use pullback val: True
-----------------------
Top results:
{'reward': 290, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 30, 'use_pullback': True}
{'reward': 341, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 4, 'pip_movement': 30, 'use_pullback': True}
{'reward': 310, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 4, 'pip_movement': 30, 'use_pullback': True}
{'reward': 531, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 4, 'pip_movement': 20, 'use_pullback': True}
{'reward': 357, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 4, 'pip_movement': 25, 'use_pullback': True}
{'reward': 672, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True}
{'reward': 456, 'ratio': 1.5, 'spre

In [11]:
# GBP_USD
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best each bar val: ' + str(best_each_bar_val))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 759.7566000000205
Best risk/reward ratio: 1.0
Best spread: 0.1
Best each bar val: False
Best n bars val: 5
Best pip movement val: 20
Best use pullback val: True
-----------------------
Top results:
{'reward': 419, 'ratio': 0.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 4, 'pip_movement': 20, 'use_pullback': True}
{'reward': 445, 'ratio': 0.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 25, 'use_pullback': True}
{'reward': 172, 'ratio': 0.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 4, 'pip_movement': 25, 'use_pullback': True}
{'reward': 759, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 20, 'use_pullback': True}
{'reward': 582, 'ratio': 1.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 25, 'use_pullback': True}
{'reward': 296, 'ratio': 0.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 5, 'pip_movement': 30, 'use_pullback': True}
{'reward': 740, 'ratio': 1.5, 'spre